# Model - YOLOv5
# type - All In One 
# Dataset - extended
# Test - VisDrone (classwise)

# 1. Install Dependencies


In [1]:
!pip install ultralytics
!pip install torch torchvision

import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 736.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 4.0 MB/s eta 0:00:00


# 2. Create dataset function (Annotation)
as the format of annotation is as - object-class,x_center,y_center,width,height (all normalized as this is input format for YOLO5 prediction)

given annotation (in pixel) - bbox_left, bbox_top, bbox_width, bbox_height, score, object_category, truncation, occlusion

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image



os.makedirs('/kaggle/working/yolo_annotations', exist_ok=True)

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Function to convert annotations
def convert_to_yolo_format(filename, annotations, img_width, img_height, given_ ,recieved_path):
    output_annotations_path = recieved_path
    with open(os.path.join(output_annotations_path, filename), 'w') as out_file:
        for annotation in annotations:
            if len(annotation) != 8:
                print(f"Error parsing annotation in {filename}: {annotation}")
                print(f"Ignoring malformed annotation: {annotation}")
                continue
            x_min, y_min, width, height, score, class_id, truncation, occlusion = map(float, annotation)
            if(class_id==0):
                continue
            # Convert to YOLO format
            center_x = x_min + (width / 2.0)
            center_y = y_min + (height / 2.0)
            center_x /= img_width
            center_y /= img_height
            width /= img_width
            height /= img_height
            class_id = class_id -1
            if(class_id == given_):
                class_id = 0
                out_file.write(f"{int(class_id)} {center_x} {center_y} {width} {height}\n")


## for train/val from extended

In [3]:
def create_files_train_val(given_):
    # Process all annotation files    # Paths to the dataset
    annotations_path = '/kaggle/input/extended-dataset/Expanded_dataset/annotations'
    images_path = '/kaggle/input/extended-dataset/Expanded_dataset/images'
    output_annotations_path = '/kaggle/working/yolo_annotations'
    
    os.makedirs( '/kaggle/working/yolo_dataset/train', exist_ok=True)
    os.makedirs( '/kaggle/working/yolo_dataset/val', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset/train')
    shutil.rmtree('/kaggle/working/yolo_dataset/val')

    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/train','images'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/val','images'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/train','labels'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/val','labels'), exist_ok=True)
    
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)


    # Split the data
    image_files = os.listdir(images_path)
    train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

    # Move files
    for file in train_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/train/images/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/train/labels/')

    for file in val_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/val/images/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/val/labels/')


## for test from visdrone

In [4]:
def create_files_test(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/images'
    os.makedirs( '/kaggle/working/yolo_dataset/test', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset/test')

        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'test/labels')
    output_images_path = os.path.join(output_dataset_path, 'test/images')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [5]:
def classes_label(class_name):
    create_files_train_val(class_name)
    create_files_test(class_name)


# 3. Import yolov8

In [6]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")

from ultralytics import YOLO

# Load a YOLOv8 model
model = YOLO('/kaggle/working/yolov8n.pt')  # You can also use yolov8m.pt, yolov8l.pt, etc.

100%|██████████| 6.23M/6.23M [00:00<00:00, 72.7MB/s]


# 4. Train Class wise



# Class 0 - pedestrian

In [7]:
classes_label(0)

In [8]:
data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['pedestrian']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)

    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='pedestrian_1')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/yolov8n.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

100%|██████████| 755k/755k [00:00<00:00, 13.8MB/s]
2024-06-27 12:54:53,187	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-27 12:54:54,846	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 286 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1341.29it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache


...

      25/25      3.56G      1.034     0.5331     0.7986        134        640: 100%|██████████| 65/65 [00:12<00:00,  5.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.62it/s]

                   all        261       2457      0.763      0.536      0.627      0.375



25 epochs completed in 0.109 hours.
Optimizer stripped from runs/detect/pedestrian_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/pedestrian_1/weights/best.pt, 6.2MB

Validating runs/detect/pedestrian_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.34it/s]


                   all        261       2457      0.745       0.54      0.624      0.383
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 11.2ms postprocess per image
Results saved to runs/detect/pedestrian_1


In [9]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:06<00:00,  2.83it/s]


                   all        261       2457      0.748      0.541      0.626      0.389
Speed: 0.3ms preprocess, 15.9ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/pedestrian_12


In [10]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/pedestrian_1/weights/best.pt')

# Define the new data.yaml file path
new_data_path = '/kaggle/working/yolo_dataset/data.yaml'

# Evaluate the model on the new dataset
test_results = model.val(data=new_data_path, split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 413 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1239.65it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:21<00:00,  4.80it/s]


                   all       1610      21006     0.0473     0.0161     0.0252    0.00972
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/val


# Class 1 - People

In [11]:
classes_label(1)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['people']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)

In [12]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='people_1')





/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/pedestrian_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=people_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 562 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1558.63it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 146 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 1586.88it/s]

...

      25/25       2.7G      1.343     0.7294     0.8053         28        640: 100%|██████████| 65/65 [00:12<00:00,  5.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.95it/s]

                   all        261        810      0.741      0.513       0.58      0.322



25 epochs completed in 0.109 hours.
Optimizer stripped from runs/detect/people_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/people_1/weights/best.pt, 6.2MB

Validating runs/detect/people_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.20it/s]


                   all        261        810      0.788      0.499      0.588      0.328
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/people_1


In [13]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 146 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  5.58it/s]


                   all        261        810      0.796      0.506      0.598      0.331
Speed: 0.2ms preprocess, 3.3ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/people_12


In [14]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/people_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 813 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1579.59it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:11<00:00,  8.63it/s]


                   all       1610       6376    0.00757     0.0177    0.00386   0.000995
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val2


## class 2 - bicycle

In [15]:
classes_label(2)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['bicycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [16]:

import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='bicycle_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/people_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=bicycle_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 816 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1650.95it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 216 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 1940.82it/s]

...

      25/25      2.33G     0.9498     0.5458     0.7807          4        640: 100%|██████████| 65/65 [00:12<00:00,  5.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.05it/s]

                   all        261        124       0.94      0.831      0.897      0.632



25 epochs completed in 0.117 hours.
Optimizer stripped from runs/detect/bicycle_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/bicycle_1/weights/best.pt, 6.2MB

Validating runs/detect/bicycle_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  4.53it/s]


                   all        261        124       0.94      0.831      0.897      0.629
Speed: 0.4ms preprocess, 1.8ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/bicycle_1


In [17]:

val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 216 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  6.89it/s]


                   all        261        124      0.941      0.831      0.896      0.641
Speed: 0.2ms preprocess, 3.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/bicycle_12


In [18]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/bicycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 1233 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<00:00, 1636.97it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:10<00:00,  9.77it/s]

                   all       1610       1302          0          0          0          0


Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val3


## class 3 - car

In [19]:
classes_label(3)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['car']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [20]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='car_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/bicycle_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=car_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 75 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1211.57it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 11 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 1345.30it/s]

...

      25/25      2.52G     0.4874     0.3068     0.7933        233        640: 100%|██████████| 65/65 [00:12<00:00,  5.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  4.93it/s]

                   all        261       3396      0.972       0.88      0.947      0.809



25 epochs completed in 0.125 hours.
Optimizer stripped from runs/detect/car_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/car_1/weights/best.pt, 6.2MB

Validating runs/detect/car_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:08<00:00,  1.06it/s]


                   all        261       3396      0.957      0.886      0.946      0.816
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/car_1


In [21]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')



Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 11 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:08<00:00,  2.00it/s]


                   all        261       3396      0.956      0.887      0.946      0.821
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/car_12


In [22]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/car_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 80 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1141.61it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:15<00:00,  6.54it/s]


                   all       1610      28074     0.0292     0.0483     0.0161    0.00511
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/val4


## Class 4 - van

In [23]:
classes_label(4)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['van']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [24]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='van_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/car_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=van_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 374 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1434.83it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 79 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 1418.24it/s]


...

      25/25      2.38G     0.4901     0.3011     0.7803         43        640: 100%|██████████| 65/65 [00:12<00:00,  5.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.87it/s]

                   all        261        707      0.977      0.942      0.981      0.853



25 epochs completed in 0.119 hours.
Optimizer stripped from runs/detect/van_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/van_1/weights/best.pt, 6.2MB

Validating runs/detect/van_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.74it/s]


                   all        261        707      0.977      0.942      0.981      0.854
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/van_1


In [25]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')



Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 79 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  4.57it/s]


                   all        261        707      0.979      0.942      0.982      0.861
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/van_12


In [26]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/van_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 442 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1432.09it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:10<00:00,  9.38it/s]


                   all       1610       5771    0.00411    0.00953    0.00213   0.000701
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val5


## Class 5 - truck

In [27]:
classes_label(5)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['truck']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [28]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='truck_1')





/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/van_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=truck_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 655 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1572.58it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      25/25      2.35G     0.5238     0.5892     0.8434         19        640: 100%|██████████| 65/65 [00:12<00:00,  5.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.43it/s]

                   all        261        275      0.918      0.876      0.943      0.821



25 epochs completed in 0.119 hours.
Optimizer stripped from runs/detect/truck_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/truck_1/weights/best.pt, 6.2MB

Validating runs/detect/truck_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.91it/s]


                   all        261        275      0.881       0.89      0.934      0.822
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/truck_1


In [29]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 158 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  5.98it/s]


                   all        261        275      0.898      0.869      0.933      0.822
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/truck_12


In [30]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/truck_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 860 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1519.95it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:10<00:00,  9.40it/s]


                   all       1610       2659    0.00213     0.0079    0.00119   0.000514
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val6


## Class 6 - tricycle

In [31]:
classes_label(6)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [32]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='tricycle_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/truck_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tricycle_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 900 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1583.92it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 233 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 2488.23it/s]

...

      25/25      2.36G     0.8327     0.8636     0.7568         12        640: 100%|██████████| 65/65 [00:11<00:00,  5.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.52it/s]

                   all        261         51      0.906      0.686      0.804      0.632



25 epochs completed in 0.118 hours.
Optimizer stripped from runs/detect/tricycle_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/tricycle_1/weights/best.pt, 6.2MB

Validating runs/detect/tricycle_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  4.25it/s]


                   all        261         51      0.919      0.725      0.803      0.639
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/tricycle_1


In [33]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 233 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  7.04it/s]


                   all        261         51      0.924      0.718      0.805       0.64
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/tricycle_12


In [34]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 1365 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<00:00, 1728.58it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:09<00:00, 10.55it/s]


                   all       1610        530    0.00127     0.0151   0.000654   0.000106
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val7


## Class 7 - awning-tricycle

In [35]:
classes_label(7)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['awning-tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [36]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='awning-tricycle_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/tricycle_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=awning-tricycle_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 909 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1801.20it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 232 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 1791.06it/s]

...

      25/25      2.35G     0.6986     0.9039     0.6973         10        640: 100%|██████████| 65/65 [00:11<00:00,  5.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.87it/s]

                   all        261         84      0.924      0.729      0.865       0.63



25 epochs completed in 0.117 hours.
Optimizer stripped from runs/detect/awning-tricycle_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/awning-tricycle_1/weights/best.pt, 6.2MB

Validating runs/detect/awning-tricycle_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  4.26it/s]


                   all        261         84      0.924      0.729      0.865       0.63
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/awning-tricycle_1


In [37]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 232 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  6.74it/s]


                   all        261         84      0.968      0.713      0.865      0.638
Speed: 0.2ms preprocess, 3.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/awning-tricycle_12


In [38]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/awning-tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 1377 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<00:00, 1780.89it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:09<00:00, 10.41it/s]


                   all       1610        599   0.000808    0.00334   0.000443   9.27e-05
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val8


## Class 8 - bus

In [39]:
classes_label(8)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['bus']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [40]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='bus_1')





/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/awning-tricycle_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=bus_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 583 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1588.84it/s]

train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 135 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 1691.24it/s]

...

      25/25      2.38G     0.4486     0.4654     0.8272         17        640: 100%|██████████| 65/65 [00:12<00:00,  5.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.70it/s]

                   all        261        336      0.898      0.836      0.909      0.826



25 epochs completed in 0.120 hours.
Optimizer stripped from runs/detect/bus_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/bus_1/weights/best.pt, 6.2MB

Validating runs/detect/bus_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.25it/s]


                   all        261        336      0.898      0.836      0.909      0.826
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/bus_1


In [41]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 135 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  5.26it/s]


                   all        261        336      0.898       0.84      0.908      0.828
Speed: 0.2ms preprocess, 4.4ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/bus_12


In [42]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/bus_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 772 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1507.41it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:10<00:00, 10.06it/s]


                   all       1610       2940      0.013     0.0616    0.00752    0.00297
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val9


## Class 9 - motor

In [43]:
classes_label(9)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['motor']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [44]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='motor_1')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/bus_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=motor_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 578 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1515.45it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 146 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 1542.63it/s]

...

      25/25      2.55G      1.125     0.7514      0.822         34        640: 100%|██████████| 65/65 [00:11<00:00,  5.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.05it/s]

                   all        261        790      0.802      0.659      0.739      0.462



25 epochs completed in 0.120 hours.
Optimizer stripped from runs/detect/motor_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/motor_1/weights/best.pt, 6.2MB

Validating runs/detect/motor_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.82it/s]


                   all        261        790        0.8      0.661      0.739      0.461
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/motor_1


In [45]:

val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')




Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 146 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  5.18it/s]


                   all        261        790       0.81      0.658       0.74      0.466
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/motor_12


In [46]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/motor_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 816 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:01<00:00, 1436.04it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:11<00:00,  9.02it/s]


                   all       1610       5845     0.0028     0.0132    0.00142   0.000357
Speed: 0.1ms preprocess, 2.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val10


## Class 10- others

In [47]:
classes_label(10)


data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['others']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)





In [48]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='others_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/motor_1/weights/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=others_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 957 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1762.49it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels... 261 images, 239 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<00:00, 2717.99it/s]


...

      25/25      2.36G     0.4873       1.51     0.5403          3        640: 100%|██████████| 65/65 [00:12<00:00,  5.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.23it/s]

                   all        261         28      0.452      0.179      0.222       0.19



25 epochs completed in 0.117 hours.
Optimizer stripped from runs/detect/others_1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/others_1/weights/best.pt, 6.2MB

Validating runs/detect/others_1/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.20it/s]


                   all        261         28      0.647      0.179       0.28       0.24
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/others_1


In [49]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 239 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  7.17it/s]


                   all        261         28      0.647      0.179       0.28      0.251
Speed: 0.2ms preprocess, 3.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/others_12


In [50]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/others_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 1610 images, 1464 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<00:00, 1805.96it/s]


val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:09<00:00, 11.02it/s]


                   all       1610        265   0.000918    0.00377   0.000466   4.66e-05
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val11
